In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('smsdata.tsv', sep='\t')
df.head()

,lab,msg,len,pun
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [2]:
df.isnull().sum()

lab    0
msg    0
len    0
pun    0
dtype: int64

In [3]:
df['lab'].value_counts()

ham     4825
spam     747
Name: lab, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split

X = df['msg']  
y = df['lab']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(3733, 7082)

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(3733, 7082)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train) # remember to use the original X_train set
X_train_tfidf.shape

(3733, 7082)

In [8]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC()

In [9]:
from sklearn.pipeline import Pipeline


text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train)  

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [10]:
predictions = text_clf.predict(X_test)

In [11]:

from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[1586    7]
 [  12  234]]


In [12]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

    accuracy                           0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [13]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.989668297988037


In [14]:
smsham = """Wait that's still not all that clear,
            were you not sure about me being sarcastic or
            that that's why x doesn't want to live with us"""

In [15]:
print(text_clf.predict([smsham]))

['ham']


In [16]:
smsspam = """Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005.
             Text FA to 87121 to receive entry question(std txt rate)T&C's apply
             08452810075over18's"""

In [17]:
print(text_clf.predict([smsspam]))

['spam']
